In [1]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
import pandas as pd
import sqlite3
import numpy as np
import sys, os
sys.path.append("../module/")
from lipinski import * 

In [2]:
pd.options.display.max_columns = 30

In [3]:
con = sqlite3.connect("../data/chembl_29.db")

In [4]:
molregno_chembl_id = pd.read_sql_query("SELECT compound_properties.*, molecule_dictionary.chembl_id, compound_structures.canonical_smiles FROM compound_properties INNER JOIN compound_structures ON compound_structures.molregno=compound_properties.molregno INNER JOIN molecule_dictionary ON molecule_dictionary.molregno=compound_properties.molregno;", con)

In [5]:
molregno_chembl_id = molregno_chembl_id.drop(['ro3_pass', 'hba', 'hbd', 'num_ro5_violations', 'full_molformula'], axis=1)

In [6]:
molregno_chembl_id = pd.merge(molregno_chembl_id.chembl_id, molregno_chembl_id)

In [7]:
molregno_chembl_id = molregno_chembl_id.drop(['molregno'], axis=1)

In [8]:
index_duplicates = molregno_chembl_id[molregno_chembl_id.canonical_smiles.duplicated()].sort_values(by=['canonical_smiles']).index

In [9]:
molregno_chembl_id = molregno_chembl_id.drop(labels=index_duplicates, axis=0)
molregno_chembl_id

,chembl_id,mw_freebase,alogp,psa,rtb,cx_most_apka,cx_most_bpka,cx_logp,cx_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,canonical_smiles
0,CHEMBL6329,341.75,2.11,84.82,3.0,6.48,NaN,3.63,2.69,ACID,341.75,3.0,24.0,0.74,341.0567,6.0,1.0,0.0,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl
1,CHEMBL6328,332.32,1.33,108.61,3.0,6.33,NaN,2.88,1.82,ACID,332.32,3.0,25.0,0.73,332.0909,7.0,1.0,0.0,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1
2,CHEMBL265667,357.80,2.27,87.98,3.0,6.33,NaN,3.70,2.64,ACID,357.80,3.0,25.0,0.75,357.0880,6.0,2.0,0.0,Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1
3,CHEMBL6362,307.31,1.46,84.82,3.0,6.33,NaN,3.02,1.97,ACID,307.31,3.0,23.0,0.74,307.0957,6.0,1.0,0.0,Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1
4,CHEMBL267864,341.75,2.11,84.82,3.0,6.33,NaN,3.63,2.57,ACID,341.75,3.0,24.0,0.74,341.0567,6.0,1.0,0.0,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084719,CHEMBL4651250,556.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,556.38,NaN,NaN,NaN,556.2055,NaN,NaN,NaN,COc1ccc([P+](CCC(=O)Nc2ccc3c(c2)B(O)OC3)(c2ccc...
2084720,CHEMBL4651251,378.30,3.13,58.36,5.0,NaN,9.33,2.96,0.43,BASE,378.30,2.0,25.0,0.84,377.1062,4.0,3.0,0.0,CNCc1ccc2c(c1)CN(C(=O)[C@H](N)Cc1ccc(Cl)cc1Cl)C2
2084721,CHEMBL4651252,356.42,2.29,90.82,7.0,9.18,9.79,1.64,0.31,BASE,356.42,2.0,26.0,0.57,356.1736,6.0,4.0,0.0,Cc1ccc(CCCNCC(O)c2cc(O)cc3c2OCC(=O)N3)cc1
2084722,CHEMBL4651253,1023.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1023.94,NaN,NaN,NaN,1023.2919,NaN,NaN,NaN,CCCCCCCCCCCCCCCCOC(=O)Oc1ccc(COP(=O)(OCc2ccc(O...


In [10]:
con_out = sqlite3.connect("../data/propriedades.db")
molregno_chembl_id.to_sql("dados", con_out)

2084588

In [10]:
ids_com_nan = molregno_chembl_id.loc[molregno_chembl_id.isnull().any(axis=1)][['chembl_id', 'canonical_smiles']]

In [11]:
ids_com_nan

,chembl_id,canonical_smiles
0,CHEMBL6329,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl
1,CHEMBL6328,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1
2,CHEMBL265667,Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1
3,CHEMBL6362,Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1
4,CHEMBL267864,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1
...,...,...
2084716,CHEMBL4651247,CN1CCN(c2cc3c(cc2F)c(=O)c(-c2nnc4sc(-c5cn(C6CC...
2084717,CHEMBL4651248,CN(C)C(=O)Oc1cccc(C(O)CNc2ccc(F)cc2)c1
2084719,CHEMBL4651250,COc1ccc([P+](CCC(=O)Nc2ccc3c(c2)B(O)OC3)(c2ccc...
2084720,CHEMBL4651251,CNCc1ccc2c(c1)CN(C(=O)[C@H](N)Cc1ccc(Cl)cc1Cl)C2


In [12]:
len(ids_com_nan)

1389314

In [15]:
def chama_atualiza_in_sql(ids_com_nan, dataframe, nome_table, con_dir, quantidades, threads_num):
  log = open("arquivo.log", "a")
  log.write("Começou\n")
  log.close()
  for i in range(quantidades):
    log = open("arquivo.log", "a")
    print(f"Parte {i + 1} de {quantidades}\n")
    log.write(f"Parte {i + 1} de {quantidades}\n")
    log.close()
    tamanho_ini = int(i * len(ids_com_nan)/quantidades)
    tamanho_fim = int((i + 1) * len(ids_com_nan)/quantidades)
    atualiza_data_frame_com_lipinski_in_sql(ids_com_nan.iloc[tamanho_ini:tamanho_fim], dataframe, nome_table, con_dir, threads_num)
  log = open("arquivo.log", "a")
  print(tamanho)
  log.write("Terminou\n")
  log.close()

In [16]:
chama_atualiza_in_sql(ids_com_nan, molregno_chembl_id, "dados", '../data/dados_att', 19*3, 19)

Parte 1 de 57

Parte 2 de 57

Parte 3 de 57

Parte 4 de 57

Parte 5 de 57

Parte 6 de 57

Parte 7 de 57

Parte 8 de 57

Parte 9 de 57

Parte 10 de 57

Parte 11 de 57

Parte 12 de 57

Parte 13 de 57

Parte 14 de 57

Parte 15 de 57

Parte 16 de 57

Parte 17 de 57

Parte 18 de 57

Parte 19 de 57

Parte 20 de 57

Parte 21 de 57

Parte 22 de 57

Parte 23 de 57

Parte 24 de 57

Parte 25 de 57

Parte 26 de 57

Parte 27 de 57

Parte 28 de 57

Parte 29 de 57

Parte 30 de 57

Parte 31 de 57

Parte 32 de 57

Parte 33 de 57

Parte 34 de 57

Parte 35 de 57

Parte 36 de 57

Parte 37 de 57

Parte 38 de 57

Parte 39 de 57

Parte 40 de 57

Parte 41 de 57

Parte 42 de 57

Parte 43 de 57

Parte 44 de 57

Parte 45 de 57

Parte 46 de 57

Parte 47 de 57

Parte 48 de 57

Parte 49 de 57

Parte 50 de 57

Parte 51 de 57

Parte 52 de 57

Parte 53 de 57

Parte 54 de 57

Parte 55 de 57

Parte 56 de 57

Parte 57 de 57

1389314


In [ ]:
%%time
quantidades = 100
threads_num = 20
for i in range(quantidades):
    print(f"Parte {i + 1} de {quantidades}")
    tamanho_ini = int(i * len(ids_com_nan)/quantidades)
    tamanho_fim = int((i + 1) * len(ids_com_nan)/quantidades)
    atualiza_data_frame_com_lipinski(ids_com_nan, molregno_chembl_id, threads_num)


In [ ]:
con_2 = sqlite3.connect('../data/dados_atualizados.db')

In [ ]:
molregno_chembl_id.to_sql('dados', con_2, if_exists='replace')

In [ ]:
molregno_chembl_id

In [ ]:
molregno_chembl_id.loc[(molregno_chembl_id.isnull().any(axis=1))]

In [ ]:
molregno_chembl_id

In [ ]:
molregno_chembl_id.query("molregno in @ids_com_nan")

In [ ]:
molregno_chembl_id.query("num_ro5_violations != num_lipinski_ro5_violations")

In [ ]:
cruzain_id = pd.read_csv('data/cruzain_id_chemb.csv')

In [ ]:
cruzain_id

In [ ]:
cruzain_id = cruzain_id["Molecule ChEMBL ID"]
cruzain_id

In [ ]:
molregno_chembl_id = molregno_chembl_id.query("chembl_id in @cruzain_id")

In [ ]:
molregno_chembl_id.head()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
molregno_chembl_id.num_lipinski_ro5_violations.unique()

In [ ]:
molregno_chembl_id.num_ro5_violations.unique()

In [ ]:
molregno_chembl_id[molregno_chembl_id.num_lipinski_ro5_violations == molregno_chembl_id.num_ro5_violations]

In [ ]:
molregno_chembl_id['ro3_pass'] = molregno_chembl_id['ro3_pass'].replace({"Y": 'True', "N": 'False'})

In [ ]:
molregno_chembl_id.query("num_lipinski_ro5_violations == 0")

In [ ]:
molregno_chembl_id.query("num_lipinski_ro5_violations == 0 & ro3_pass == 'True'")

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [ ]:
prep_clust = pd.get_dummies(molregno_chembl_id.query("num_lipinski_ro5_violations == 0"), columns=["molecular_species"])
tab_usada = prep_clust
prep_clust = prep_clust.drop(['chembl_id','molregno', "full_molformula", "canonical_smiles"], axis=1)


In [ ]:
prep_clust.head()

In [ ]:
prep_clust = prep_clust.replace({np.nan: "7"})
prep_clust.head()

In [ ]:
x = preprocessing.normalize(prep_clust, norm='l1')
cluster = KMeans(n_clusters=3, random_state=5)
cluster.fit(x).labels_

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(x=prep_clust[['mw_freebase']], y=prep_clust.psa, c=cluster.fit(x).labels_, s=50)
plt.xlabel('mw_freebase', fontsize=18)
plt.ylabel('psa', fontsize=18)
plt.show()

In [ ]:
tab_usada['grupos_cluster_knn'] = cluster.fit(x).labels_
tab_usada.drop(["num_ro5_violations", "ro3_pass", "num_lipinski_ro5_violations"], axis=1).sort_values(by='grupos_cluster_knn')

In [ ]:
sns.pairplot(tab_usada[['mw_freebase', 'psa', 'grupos_cluster_knn']], hue='grupos_cluster_knn')


In [ ]:
tab_usada['grupos_cluster_knn'] = cluster.fit(x).labels_
tab_usada.drop(["num_ro5_violations", "ro3_pass", "num_lipinski_ro5_violations"], axis=1).sort_values(by='grupos_cluster_knn')